<a href="https://colab.research.google.com/github/sunanda239u/autonomous-agricultural-drone/blob/main/sentiment_analysis_of_amazon_review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

all imports

In [ ]:
import pandas as pd
import numpy as np
import bz2
import os

import re
import gc

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.python.keras import models, layers, optimizers

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

created a functin to assign labels and comments to the given dataset

In [ ]:
def assign_labels_and_comments(file):
    labels = []
    comments = []
    for line in bz2.BZ2File(file):
        x = line.decode("utf-8")
        labels.append(int(x[9]) - 1)
        comments.append(x[10:].strip())
    return np.array(labels), comments

assigned labels and dataset to both test and train dataset

In [ ]:
 test_labels, test_comments = assign_labels_and_comments('/content/test.ft.txt.bz2')

NameError: ignored

In [ ]:
train_labels, train_comments = assign_labels_and_comments('/content/train.ft.txt.bz2')

function to preprocess the given dataset

In [ ]:
def processed_comments(texts):
    processed_comments = []
    for text in texts:
        lower = text.lower()
        no_punctuation = not_alphanumeric.sub(r' ', lower)
        no_non_ascii = not_ascii.sub(r'', no_punctuation)
        processed_comments.append(no_non_ascii)
    return processed_comments

In [ ]:
not_alphanumeric = re.compile(r'[\W]')
not_ascii = re.compile(r'[^a-z0-1\s]')

preprocessed both the dataset

In [ ]:
test_comments = processed_comments(test_comments)

In [ ]:
train_comments = processed_comments(train_comments)

spliting the train dataset into train and validation dataset

In [ ]:
train_comments, val_comments, train_labels, val_labels = train_test_split(train_comments, train_labels, random_state=42, test_size=0.2)

converting features to tokens

In [ ]:
maximum_features = 14000
tokenizer = Tokenizer(num_words=maximum_features)
tokenizer.fit_on_texts(train_comments)
train_comments = tokenizer.texts_to_sequences(train_comments)
val_comments = tokenizer.texts_to_sequences(val_comments)
test_comments = tokenizer.texts_to_sequences(test_comments)

In [ ]:
maximum_length = max(len(train_ex) for train_ex in train_comments)
train_comments_pad = pad_sequences(train_comments, maxlen=maximum_length)
val_comments_pad = pad_sequences(val_comments, maxlen=maximum_length)
test_comments_pad = pad_sequences(test_comments, maxlen=maximum_length)
del train_comments, val_comments, test_comments
gc.collect()

training CNN model

In [ ]:
def cnn_model():
    sequences = layers.Input(shape=(maximum_length,))
    embedded = layers.Embedding(maximum_features, 64)(sequences)
    x = layers.Conv1D(64, 3, activation='relu')(embedded)
    x = tf.keras.layers.BatchNormalization()(x)
    x = layers.MaxPool1D(3)(x)
    x = layers.Conv1D(64, 5, activation='relu')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = layers.MaxPool1D(5)(x)
    x = layers.Conv1D(64, 5, activation='relu')(x)
    x = layers.GlobalMaxPool1D()(x)
    x = layers.Flatten()(x)
    x = layers.Dense(100, activation='relu')(x)
    predictions = layers.Dense(1, activation='sigmoid')(x)
    model = models.Model(inputs=sequences, outputs=predictions)
    model.compile(
        optimizer='rmsprop',
        loss='binary_crossentropy',
        metrics=['binary_accuracy']
    )
    return model

model = cnn_model()

In [ ]:
model.fit(
    train_comments_pad,
    train_labels,
    batch_size=128,
    epochs=3,
    validation_data=(val_comments_pad, val_labels))

In [ ]:
finding accuracy of CNN model

In [ ]:
cnn_preds = model.predict(test_comments_pad)

accuracy_cnn = accuracy_score(test_labels, 1 * (cnn_preds > 0.5))
f1_cnn = f1_score(test_labels, 1 * (cnn_preds > 0.5))
rocauc_cnn = roc_auc_score(test_labels, cnn_preds)

print('Accuracy score of the CNN Model: {:0.3}'.format(accuracy_cnn))
print('F1 score of the CNN Model: {:0.3}'.format(f1_cnn))
print('ROC AUC score of the CNN Model: {:0.3}'.format(rocauc_cnn))

In [ ]:
gc.collect()

Creating RNN model

In [ ]:
def rnn_model():
    sequences = layers.Input(shape=(maximum_length,))
    embedded = layers.Embedding(maximum_features, 64)(sequences)
    x = layers.CuDNNGRU(128, return_sequences=True)(embedded)
    x = layers.CuDNNGRU(128)(x)
    x = layers.Dense(32, activation='relu')(x)
    x = layers.Dense(100, activation='relu')(x)
    predictions = layers.Dense(1, activation='sigmoid')(x)
    model = models.Model(inputs=sequences, outputs=predictions)
    model.compile(
        optimizer='rmsprop',
        loss='binary_crossentropy',
        metrics=['binary_accuracy']
    )
    return model

rnn_model = rnn_model()

In [ ]:
rnn_model.fit(train_comments_pad,
    train_labels,
    batch_size=128,
    epochs=3,
    validation_data=(val_comments_pad, val_labels))

Finding accuracy of RNN model

In [ ]:
rnn_preds = rnn_model.predict(test_comments_pad)

accuracy_rnn = accuracy_score(test_labels, 1 * (rnn_preds > 0.5))
f1_rnn = f1_score(test_labels, 1 * (rnn_preds > 0.5))
rocauc_rnn = roc_auc_score(test_labels, rnn_preds)

print('Accuracy score of the RNN Model: {:0.3}'.format(accuracy_rnn))
print('F1 score of the RNN Model: {:0.3}'.format(f1_rnn))
print('ROC AUC score of the RNN Model: {:0.3}'.format(rocauc_rnn))

In [ ]:
model_acc_dict = {}
model_f1_dict = {}
model_rocauc_dict = {}

In [ ]:
model_acc_dict['Convolutional_Neural_Network'] = accuracy_cnn
model_f1_dict['Convolutional_Neural_Network'] = f1_cnn
model_rocauc_dict['Convolutional_Neural_Network'] = rocauc_cnn

model_acc_dict['Recurrent_Neural_Network'] = accuracy_rnn
model_f1_dict['Recurrent_Neural_Network'] = f1_rnn
model_rocauc_dict['Recurrent_Neural_Network'] = rocauc_rnn

In [ ]:
created 3 custom functions to display the comparison graph

In [ ]:
def visualize_model_f1_scores(model_dict):
    model_f1_scores_df = pd.DataFrame(columns=['Model', 'F1 Score'])
    model_f1_scores_df['Model'] = model_dict.keys()
    model_f1_scores_df['F1 Score'] = model_dict.values()
    model_f1_scores_df.sort_values('F1 Score', inplace=True, ascending=False)

    plt.figure(figsize=(6,4),)
    plt.ylabel("Models", fontsize=16)
    plt.xlabel("F1 Score", fontsize=16)
    plt.title("Model F1 Scores", fontsize=22)
    sns.barplot(y = pd.to_numeric(model_f1_scores_df['F1 Score']), x = model_f1_scores_df['Model'], palette='magma')
    plt.show()

    print("The Models have the following F1 Scores:")
    print(f"{model_f1_scores_df.iloc[:2, ]}")

In [ ]:
def visualize_model_accuracies(model_dict):
    model_accuracies_df = pd.DataFrame(columns=['Model', 'Accuracy'])
    model_accuracies_df['Model'] = model_dict.keys()
    model_accuracies_df['Accuracy'] = model_dict.values()
    model_accuracies_df.sort_values('Accuracy', inplace=True, ascending=False)

    plt.figure(figsize=(6,4),)
    plt.ylabel("Models", fontsize=16)
    plt.xlabel("Accuracy", fontsize=16)
    plt.title("Model Accuracies", fontsize=22)
    sns.barplot(y = pd.to_numeric(model_accuracies_df['Accuracy']), x = model_accuracies_df['Model'], palette='magma')
    plt.show()

    print("The Models have the following accuracies :")
    print(f"{model_accuracies_df.iloc[:2, ]}")

In [ ]:
def visualize_model_rocauc_scores(model_dict):
    model_rocauc_scores_df = pd.DataFrame(columns=['Model', 'ROC AUC'])
    model_rocauc_scores_df['Model'] = model_dict.keys()
    model_rocauc_scores_df['ROC AUC'] = model_dict.values()
    model_rocauc_scores_df.sort_values('ROC AUC', inplace=True, ascending=False)

    plt.figure(figsize=(6,4),)
    plt.ylabel("Models", fontsize=16)
    plt.xlabel("ROC AUC", fontsize=16)
    plt.title("Model ROC AUC", fontsize=22)
    sns.barplot(y = pd.to_numeric(model_rocauc_scores_df['ROC AUC']), x = model_rocauc_scores_df['Model'], palette='magma')
    plt.show()

    print("The Models have the following ROC AUC Scores :")
    print(f"{model_rocauc_scores_df.iloc[:2, ]}")

called the functions to display the visual comparison

In [ ]:
visualize_model_accuracies(model_acc_dict)

In [ ]:
visualize_model_f1_scores(model_f1_dict)

In [ ]:
visualize_model_rocauc_scores(model_rocauc_dict)